<a href="https://colab.research.google.com/github/UV1L/MyFirstNeutralNetwork/blob/main/NeutralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Моя первая нейронка](https://)**

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Скачаем данные с соревнования https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

Запишем данные из csv в переменную

In [ ]:
import pandas as pd


csv_train_data = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
comments = csv_train_data["comment_text"]
y_train = csv_train_data[["toxic",
                          "severe_toxic",
                          "obscene",
                          "threat",
                          "insult",
                          "identity_hate"
                          ]]

Создаем токенизатор и обучаем его

Максимальное число слов = 10000

In [ ]:
from tensorflow.keras.preprocessing.text import *


my_tokenizer = Tokenizer(num_words=10000)

Обучаем токенизатор на тестовых комментариях, которые мы до этого получили

In [ ]:
my_tokenizer.fit_on_texts(comments)
my_tokenizer.word_index

Получаем число символов, соотвествующих каждому слову в тестовых комментариях

In [ ]:
sequences = my_tokenizer.texts_to_sequences(comments)

Вот пример одного такого комментария:

In [ ]:
sequences[1]

Получаем численное представление тестовых данных с максимальной длиной комментария в 50 символов

In [ ]:
from keras.preprocessing.sequence import *


x_train = pad_sequences(sequences, 50)

In [ ]:
x_train[::]

Далее создаем модель последоваельной сети lstm

In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SpatialDropout1D
from keras.layers import LSTM
from keras.datasets import imdb

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(10000, 128, input_length=50))
model.add(SpatialDropout1D(0.5))

1.   Добавляем слой долго-краткосрочной памяти
2.   Добавляем полносвязный слой из 1 элемента для классификации, в качестве функции активации будем использовать сигмоидальную функцию
3.   Компилируем модель



Компилируем

In [ ]:
model.add(LSTM(40, return_sequences=True))
model.add(LSTM(40))
model.add(Dense(6, activation="sigmoid"))
model.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

Делаем чекпоинт c флагом save_best_only для сохранения самой качественной эпохи

In [ ]:
from keras.callbacks import *


h5 = "model.h5"
model_checkpoint = ModelCheckpoint(h5,
                                   monitor="val_accuracy",
                                   save_best_only=True,
                                   verbose=0.2)


Обучаем нейронную сеть (данные для обучения, ответы к данным для обучения, число эпох обучения, количество рецензий после анализа которого будут изменены веса,
куда записывать колбек)

In [ ]:
model.fit(x_train,
          y_train,
          epochs=3,
          batch_size=256,
          validation_split=0.2,
          callbacks=[model_checkpoint])

Загрузим тестовые комментарии и объединим их с таблицей ответов по id

In [ ]:
comments_test = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")
comments_labels = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip")

comments_test_with_labels = pd.merge(comments_test, comments_labels, on="id")
toxic = comments_test_with_labels["toxic"]
comments_test_without_toxic = comments_test_with_labels
test_ids = comments_test.id.values

Преобразуем в числовое представление

In [ ]:
sequences_test = my_tokenizer.texts_to_sequences(comments_test_without_toxic["comment_text"])

x_test = pad_sequences(sequences_test, maxlen=50)
y_test = comments_test_without_toxic[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

Протестируем нейросеть:

In [ ]:
model.load_weights(h5)
model.evaluate(x_test, y_test, verbose=0.2)

In [ ]:
predict = model.predict(x_test)
predict.shape

In [ ]:
test_ids.shape
test_ids.reshape(-1, 1)
sub_df = pd.DataFrame(predict, columns=csv_train_data.columns[2:])
sub_df["id"] = test_ids
sub_df

In [ ]:
columns = sub_df.columns.tolist()
columns = columns[-1:] + columns[:-1]
sub_df = sub_df[columns]

In [ ]:
my_test_non_toxic_comment = "Hello world!"
my_test_toxic_comment = "I will kill u!"
first_sequence = my_tokenizer.texts_to_sequences([my_test_non_toxic_comment, my_test_toxic_comment])
pad_seq = pad_sequences(first_sequence, maxlen=50)
result = model.predict(pad_seq)
result

Запишем результаты в csv

In [ ]:
sub_df.to_csv("submission.csv", index=False, header=True)